# Understanding the Use of Claudette Tool Loop and build JSON agent based on the evaluator corrector pattern

In [ ]:
from claudette.core import *
from fastcore.utils import *
import fastcore.all as fc, re, math, itertools, functools, numpy as np, types, typing, dataclasses, matplotlib.pyplot as plt, collections, regex
from regex import search
from collections import Counter
from collections.abc import Iterable
from pydantic import BaseModel, Field, field_validator
import uuid
from typing import Literal, Union, List, Dict, Any
from toolslm.funccall import get_schema, mk_ns, call_func
from rdflib.plugins.parsers.notation3 import BadSyntax
from rdflib import Graph, URIRef
from pyshacl import validate
import json
import httpx

**AI Prompt**

Hi solveit! I am Chuck, and I am working with you today! I'm know some Python, experienced with semantic web technologies like OWL,RDF and SPARQL, and I enjoy learning new things. I am interested in understanding Claudette, the Anthropic API, Structured Data and Function calling.How When writing code for me, please follow fast.ai coding style: avoid comments; place single line if/loop/etc constructs on the same line; use concise variable names where appropriate. Follow the style you will see in my messages, as we work on this dialogue together. Are you ready?

**AI Response**

Hi Chuck! Great to meet you. I see you have a strong background in semantic technologies and Python, which will be really helpful for exploring Claudette and structured data handling.

Before we dive in, could you tell me which aspect you'd like to explore first:

1. Claudette's structured data capabilities
2. Function calling patterns
3. Integration with semantic web technologies
4. Something else?

This will help me tailor the examples and explanations to your interests and expertise level.

From the claudette documentation: https://claudette.answer.ai/toolloop.html
Anthropic provides an [interesting example](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/customer_service_agent.ipynb) of using tools to mock up a hypothetical ordering system. We’re going to take it a step further, and show how we can dramatically simplify the process, whilst completing more complex tasks.

We’ll start by defining the same mock customer/order data as in Anthropic’s example, plus create a entity relationship between customers and orders:

In [ ]:
model = models[-1]
model

'claude-3-5-haiku-20241022'

In [ ]:
# Define a model function that returns models
def default_model(): return models[-1]  # Claude 3.5 Haiku

In [ ]:
import os
os.environ['ANTHROPIC_LOG'] = 'debug'

In [ ]:
orders = {
    "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
    "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
    "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

customers = {
    "C1": dict(name="John Doe", email="[email protected]", phone="123-456-7890",
               orders=[orders['O1'], orders['O2']]),
    "C2": dict(name="Jane Smith", email="[email protected]", phone="987-654-3210",
               orders=[orders['O3']])
}

We can now define the same functions from the original example – but note that we don’t need to manually create the large JSON schema, since Claudette handles all that for us automatically from the functions directly. We’ll add some extra functionality to update order details when cancelling too.

In [ ]:
def get_customer_info(
    customer_id:str # ID of the customer
): # Customer's name, email, phone number, and list of orders
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")

def get_order_details(
    order_id:str # ID of the order
): # Order's ID, product name, quantity, price, and order status
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")

def cancel_order(
    order_id:str # ID of the order to cancel
)->bool: # True if the cancellation is successful
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True

In [ ]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = Chat(model, tools=tools)

In [ ]:
r = chat('Can you tell me the email address for customer C1?')
print(r.stop_reason)
r.content

- Retrieving customer C1
tool_use


[TextBlock(citations=None, text="I'll retrieve the customer information for customer ID C1 using the get_customer_info function.", type='text'),
 ToolUseBlock(id='toolu_01T4TvmGELVNV9Su6U97V8TU', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

In [ ]:
r = chat()
contents(r)

'The email address for customer C1 (John Doe) is [email protected]. Please note that the email appears to be masked in the system for privacy reasons.\n\nIs there anything else I can help you with regarding this customer?'

Let’s consider a more complex case than in the original example – what happens if a customer wants to cancel all of their orders?

In [ ]:
chat = Chat(model, tools=tools)
r = chat('Please cancel all orders for customer C1 for me.')
print(r.stop_reason)
r.content

- Retrieving customer C1
tool_use


[TextBlock(citations=None, text="I'll help you cancel all orders for customer C1. To do this, I'll first retrieve the customer's information to get their order details, and then cancel each order.", type='text'),
 ToolUseBlock(id='toolu_014jZC7qFkt3YHPWKozFmmt2', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

**AI Prompt**

This is an example of a Toolloop from the Claudette documentation. I don't really understand the example. Can you walk me through and help me to understand.

**AI Response**

I'll help you understand this example step by step. Let's break down what's happening:

1. First, let's look at the data structure:
- There's a dictionary of `orders` with order IDs (O1, O2, O3)
- There's a dictionary of `customers` with customer IDs (C1, C2)
- Each customer has a list of their orders

Would you like me to explain this data structure in more detail before we move on to how the tools and chat interaction work?

**AI Prompt**

So these are dictionaries of dictionaries?

**AI Response**

Yes, exactly! Let's look at the structure more precisely:

```python
orders = {
    "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
    # Each order is a dictionary with details about that order
}

customers = {
    "C1": dict(name="John Doe", email="...", phone="...",
               orders=[orders['O1'], orders['O2']]),
    # Each customer is a dictionary that includes a list of their order dictionaries
}
```

The interesting part is how the `customers` dictionary links to the `orders` - notice how each customer's `orders` field contains actual order dictionaries, not just order IDs.

Would you like me to explain how these dictionaries are used by the tools next?

**AI Prompt**

Ok this would be equivalent to multiple tables in a relational database.

**AI Response**

Exactly! In a relational database, this would typically be structured as:

1. A `customers` table with columns:
   - customer_id (PK)
   - name
   - email
   - phone

2. An `orders` table with columns:
   - order_id (PK)
   - product
   - quantity
   - price
   - status
   - customer_id (FK)

The key difference here is that instead of using foreign keys to establish relationships, the Python dictionaries are using direct object references - embedding the entire order objects within the customer dictionary.

Would you like to see how the tools (`get_customer_info`, `get_order_details`, and `cancel_order`) act like database queries on these dictionaries?

**AI Prompt**

Yes. Let's walk through the convenience functions.

**AI Response**

Let's look at each function one at a time, starting with `get_customer_info`:

```python
def get_customer_info(customer_id:str):
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")
```

This function is like a simple SELECT query - it uses Python's dict.get() method to safely retrieve a customer's entire record, including their embedded orders. If the customer_id doesn't exist, it returns "Customer not found" instead of raising an error.

Would you like me to explain how this function is used in the chat example, or should we look at the next function first?

**AI Prompt**

Let's go through the functions then move to the Chat example.

**AI Response**

OK, let's look at the next function, `get_order_details`:

```python
def get_order_details(order_id:str):
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")
```

This is similar to `get_customer_info` but for orders - it looks up an order by ID in the orders dictionary.

Finally, we have `cancel_order`:

```python
def cancel_order(order_id:str)->bool:
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True
```

This one actually modifies the data - it changes the status of an order to 'Cancelled' and returns True if successful, False if the order wasn't found.

Would you like to see how these functions are used together in the Chat example now?

In [ ]:
get_schema(get_order_details)

{'name': 'get_order_details',
 'description': 'Retrieves the details of a specific order based on the order ID',
 'input_schema': {'type': 'object',
  'properties': {'order_id': {'type': 'string',
    'description': 'ID of the order'}},
  'title': None,
  'required': ['order_id']}}

In [ ]:
get_schema(cancel_order)

{'name': 'cancel_order',
 'description': 'Cancels an order based on the provided order ID\n\nReturns:\n- type: boolean',
 'input_schema': {'type': 'object',
  'properties': {'order_id': {'type': 'string',
    'description': 'ID of the order to cancel'}},
  'title': None,
  'required': ['order_id']}}

In [ ]:
get_schema(get_customer_info)

{'name': 'get_customer_info',
 'description': "Retrieves a customer's information and their orders based on the customer ID",
 'input_schema': {'type': 'object',
  'properties': {'customer_id': {'type': 'string',
    'description': 'ID of the customer'}},
  'title': None,
  'required': ['customer_id']}}

**AI Prompt**

Ok. I grabbed the schema representation of the functions so it is more clear when you go through the Chat example.

**AI Response**

Perfect! Now let's look at how these functions are used in the Chat example. The first part sets up the chat with the tools:

```python
tools = [get_customer_info, get_order_details, cancel_order]
chat = Chat(model, tools=tools)
```

Then there's this interaction:

```python
r = chat('Can you tell me the email address for customer C1?')
print(r.stop_reason)
```

When this runs, we see:
- Output: "- Retrieving customer C1"
- stop_reason: "tool_use"

This shows that Claude recognized it needed to use a tool to answer the question. Looking at the schemas, it knew to use `get_customer_info`.

Would you like me to explain what happens in the next step of this interaction?

**AI Prompt**

Yes! Please continue!

**AI Response**

Great! Let's look at what happens next. After the first response indicating tool use, we call:

```python
r = chat()
```

This continues the conversation, and Claude processes the result from `get_customer_info` to answer the original question. The response was:
"The email address for customer C1 (John Doe) is [email protected]..."

This shows how the Chat interaction works in steps:
1. Claude recognizes it needs customer info
2. It uses the appropriate tool
3. It receives the data
4. It formats a human-readable response

Would you like to see how this gets more interesting with the more complex example of canceling all orders for a customer?

**AI Prompt**

Yes. Explain the complex tool use chain.

**AI Response**

Let's look at the more complex example where multiple tools need to be used in sequence:

```python
chat = Chat(model, tools=tools)
r = chat('Please cancel all orders for customer C1 for me.')
```

In this case, Claude needs to:
1. First get the customer's info to find their orders (using `get_customer_info`)
2. Then cancel each order (using `cancel_order` multiple times)

Looking at the first response, we see:
- Output: "- Retrieving customer C1"
- stop_reason: "tool_use"

This is just the first step. Would you like to see what happens when we continue this interaction to handle all the orders?

**AI Prompt**

Yes. Explain the next step.

**AI Response**

Let's continue with the next step. When we call `chat()` again, Claude will receive the customer information for C1, which includes two orders: O1 and O2.

Let's see what happens when we run:
```python
r = chat()
```

At this point, Claude should recognize it needs to cancel each order individually using the `cancel_order` tool. Looking at the response, it should start the process of canceling the first order.

Would you like me to show you the actual response and explain how Claude handles canceling multiple orders one by one?

**AI Prompt**

Yes. Interpet the actual response and explain how claude handles the chain.

**AI Response**

Let's run through it and look at the actual responses:

```python
r = chat()
```

When we run this, Claude:
1. Has the customer info from C1 (which has orders O1 and O2)
2. Calls `cancel_order` for O1
3. We'll see output: "- Cancelling order O1"
4. The stop_reason will be "tool_use" again

This is because Claude is working through a chain of operations, and can only execute one tool call at a time. Each `chat()` call advances the chain one step.

Would you like me to show what happens in the next step when we call `chat()` again to handle the second order (O2)?

**AI Prompt**

So, I'm a bit confused. I see 
print(r.stop_reason)
r.content
after the r.chat('Please cancel all orders for customer C1 for me.'). Did this not finish through the toolchain?

**AI Response**

Ah, good catch! Let's look at that specific interaction more carefully:

```python
r = chat('Please cancel all orders for customer C1 for me.')
print(r.stop_reason)  # Outputs: tool_use
r.content  # Shows: Claude planning to get customer info and using get_customer_info tool
```

You're right - this first response only initiates the chain. To complete the entire operation (getting customer info and canceling all orders), we need to use Claudette's `toolloop` method instead of manual `chat()` calls. This would automate the entire chain of operations.

Would you like me to show you how to use `toolloop` to handle this multi-step process more efficiently?

In [ ]:
chat = Chat(model, tools=tools)
r = chat.toolloop('Can you tell me the email address for customer C1?')
r

- Retrieving customer C1


The email address for customer C1 (John Doe) is [email protected]. Please note that the email appears to be masked in the system for privacy reasons.

Is there anything else I can help you with regarding this customer?

<details>

- id: `msg_01TPX1guFqVJANiDRE3T9srB`
- content: `[{'citations': None, 'text': 'The email address for customer C1 (John Doe) is [email protected]. Please note that the email appears to be masked in the system for privacy reasons.\n\nIs there anything else I can help you with regarding this customer?', 'type': 'text'}]`
- model: `claude-3-5-haiku-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 741, 'output_tokens': 52}`

</details>

In [ ]:
chat = Chat(model, tools=tools)
r = chat.toolloop('Please cancel all orders for customer C1 for me.', trace_func=print)
r

- Retrieving customer C1
[{'role': 'user', 'content': 'Please cancel all orders for customer C1 for me.'}, {'role': 'assistant', 'content': [TextBlock(citations=None, text="I'll help you cancel all orders for customer C1. To do this, I'll first retrieve the customer's information to get their order details, and then cancel each order.", type='text'), ToolUseBlock(id='toolu_01A7LuVHhrP4RQyg4ZABRAjA', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01A7LuVHhrP4RQyg4ZABRAjA', 'content': "{'name': 'John Doe', 'email': '[email\\xa0protected]', 'phone': '123-456-7890', 'orders': [{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Shipped'}, {'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Processing'}]}"}]}]
- Cancelling order O1
[{'role': 'assistant', 'content': [TextBlock(citations=None, text="I see that customer C1 (John Doe) has two

I've successfully cancelled both orders (O1 and O2) for customer C1. Both cancellations returned 'True', which indicates they were processed successfully. Is there anything else I can help you with?

<details>

- id: `msg_014Nvbi1EWuv6hMdUf2LmzLy`
- content: `[{'citations': None, 'text': "I've successfully cancelled both orders (O1 and O2) for customer C1. Both cancellations returned 'True', which indicates they were processed successfully. Is there anything else I can help you with?", 'type': 'text'}]`
- model: `claude-3-5-haiku-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 926, 'output_tokens': 49}`

</details>

**AI Prompt**

Ok. I think I now have added the toolloop example correctly?

**AI Response**

Yes! Now we can see the complete toolloop in action. Let's break down what happened in the order cancellation example with `toolloop`:

1. First call: `get_customer_info`
```
- Retrieving customer C1
```

2. After getting customer info, Claude automatically proceeds to cancel O1:
```
- Cancelling order O1
```

3. Then automatically moves to O2:
```
- Cancelling order O2
```

4. Finally provides a summary:
```
"I've successfully cancelled both orders (O1 and O2) for customer C1..."
```

The key difference is that `toolloop` automatically handles the entire chain of operations without needing separate `chat()` calls. Would you like me to explain how we can use the `trace_func` parameter to debug these tool chains?

**AI Prompt**

Yes. What is trace_func and how do I use it to debug the chain?

**AI Response**

Let's look at how `trace_func` works. In the example, we used:

```python
r = chat.toolloop('Please cancel all orders for customer C1 for me.', trace_func=print)
```

The `trace_func` parameter lets us see the entire conversation between Claude and the tools. When we use `print` as the trace function, we see the full chain of messages:

1. User's initial request
2. Claude's plan to get customer info
3. The tool response with customer data
4. Claude's decision to cancel O1
5. The response from canceling O1
6. Claude's move to cancel O2
7. The final response from canceling O2

Would you like me to break down one of these trace messages in detail to show you what information it contains?

## Example Agentic Validator-Corrector Pattern
Impliment Agentic pattern using Claudette toolloop.

- Use Python JSON to parse JSON object
- Detect JSON syntax errors from the Python JSON Parser
- Report them in a structured way
- Use Claude to automatically fix these errors
- Call Python JSON tool to parse corrected JSON
- Continue until JSON successfully parsed

**AI Prompt**

Ok. I would like to create a new toolloop example based on JSON. I want to do this in a solve it way, just like the original customer info example. Can you help me?

**AI Response**

I'll help you create a JSON validation and correction toolloop example. Let's break this down into steps:

First, let's create some simple functions to handle JSON operations. Would you like me to suggest the basic tools we'll need? They would be similar to the customer example, but focused on:

1. A function to validate JSON
2. A function to parse JSON
3. A function to fix JSON errors

Would you like me to show you how to create these basic tools first?

**AI Prompt**

Yes. Let's make the basic tools first.

**AI Response**

Let's create the first tool - a JSON validator. We'll make it return both success/failure and any error message:

```python
def validate_json(
    json_str:str # JSON string to validate
) -> dict: # Dictionary with validation result and error message if any
    "Validates if a string is valid JSON"
    try:
        json.loads(json_str)
        return {"valid": True, "error": None}
    except json.JSONDecodeError as e:
        return {"valid": False, "error": str(e)}
```

Would you like to test this function with a simple example before we create the next tool?

In [ ]:
def validate_json(
    json_str:str # JSON string to validate
) -> dict: # Dictionary with validation result and error message if any
    "Validates if a string is valid JSON"
    try:
        json.loads(json_str)
        return {"valid": True, "error": None}
    except json.JSONDecodeError as e:
        return {"valid": False, "error": str(e)}

**AI Prompt**

Yes. Let's test this function first.

**AI Response**

Let's test the validate_json function with both valid and invalid JSON:

```python
# Test with valid JSON
valid_json = '{"name": "John", "age": 30}'
print("Valid JSON test:", validate_json(valid_json))

# Test with invalid JSON (missing quote)
invalid_json = '{"name": John, "age": 30}'
print("Invalid JSON test:", validate_json(invalid_json))
```

Let's run this and see how our validator handles both cases. Would you like to try it with these examples or would you prefer to test with different JSON?

In [ ]:
# Test with valid JSON
valid_json = '{"name": "John", "age": 30}'
print("Valid JSON test:", validate_json(valid_json))

# Test with invalid JSON (missing quote)
invalid_json = '{"name": John, "age": 30}'
print("Invalid JSON test:", validate_json(invalid_json))

Valid JSON test: {'valid': True, 'error': None}
Invalid JSON test: {'valid': False, 'error': 'Expecting value: line 1 column 10 (char 9)'}


**AI Prompt**

Ok. This seems to work.

**AI Response**

Great! Now let's create our second tool - a JSON parser that will return the parsed data if successful:

```python
def parse_json(
    json_str:str # JSON string to parse
) -> dict: # Parsed JSON data or error message
    "Parses a JSON string into a Python dictionary"
    try:
        return {"success": True, "data": json.loads(json_str)}
    except json.JSONDecodeError as e:
        return {"success": False, "data": str(e)}
```

Would you like to test this parser function before we create our final tool for fixing JSON errors?

In [ ]:
def parse_json(
    json_str:str # JSON string to parse
) -> dict: # Parsed JSON data or error message
    "Parses a JSON string into a Python dictionary"
    try:
        return {"success": True, "data": json.loads(json_str)}
    except json.JSONDecodeError as e:
        return {"success": False, "data": str(e)}

**AI Prompt**

So, it seems that parse_json and validate_json are the same function? Does this make sense.

**AI Response**

Ah, good catch! You're right - these functions are doing almost the same thing, just returning the results in slightly different formats. We should consolidate this into a single, more useful function.

Let's modify our approach:

```python
def check_json(
    json_str:str # JSON string to validate and parse
) -> dict: # Dictionary with validation status, error (if any), and parsed data (if valid)
    "Validates and parses JSON, returning status, any error message, and parsed data if valid"
    try:
        data = json.loads(json_str)
        return {"valid": True, "error": None, "data": data}
    except json.JSONDecodeError as e:
        return {"valid": False, "error": str(e), "data": None}
```

This combines both validation and parsing in one function. Would you like to test this revised version?

In [ ]:
def check_json(
    json_str:str # JSON string to validate and parse
) -> dict: # Dictionary with validation status, error (if any), and parsed data (if valid)
    "Validates and parses JSON, returning status, any error message, and parsed data if valid"
    try:
        data = json.loads(json_str)
        return {"valid": True, "error": None, "data": data}
    except json.JSONDecodeError as e:
        return {"valid": False, "error": str(e), "data": None}